In [1]:
from keras.layers import Input, Activation,Dense, Dropout, TimeDistributed
from keras.layers.recurrent import LSTM 
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
import numpy as np
from keras.models import Model

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [2]:
batch_size = 64
STEP_SIZE = 15
INPUT_SIZE = 20
CELL_SIZE = 25

In [3]:
inputs = Input((STEP_SIZE, INPUT_SIZE))

In [5]:
x = LSTM(CELL_SIZE, input_shape=(STEP_SIZE, INPUT_SIZE))(inputs)

In [6]:
x = Dropout(0.25)(x)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
x = RepeatVector(STEP_SIZE)(x)

In [8]:
# STEP_SIZE, CELL_SIZE -> STEP_SIZE, NERVE_NUM
# 当return_sequences=True时，会输出时间序列
# STEP_SIZE代表时间序列，CELL_SIZE代表每一个时间序列的输出
x = LSTM(CELL_SIZE, return_sequences=True)(x)

In [9]:
# 对每一个STEP进行全连接
x = TimeDistributed(Dense(INPUT_SIZE))(x)
x = Dropout(0.5)(x)
x = Activation('softmax')(x)

In [10]:
model = Model(inputs, x)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15, 20)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                4600      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 15, 25)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 15, 25)            5100      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 20)            520       
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 20)            0         
__________

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [13]:
def batch_gen(batch_size=32, seq_len=10, max_no=100):
    while True:
        x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
        y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

        X = np.random.randint(max_no, size=(batch_size, seq_len))
        Y = np.sort(X, axis=1)

        for ind, batch in enumerate(X):
            for j, elem in enumerate(batch):
                x[ind, j, elem] = 1

        for ind, batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1
        yield x, y 

In [14]:
def encode(X, seq_len, vocab_size):
    x = np.zeros((len(X), seq_len, vocab_size), dtype=np.float32)
    for ind, batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x 

In [15]:
for ind, (X, Y) in enumerate(batch_gen(batch_size, STEP_SIZE, INPUT_SIZE)):
    loss, acc = model.train_on_batch(X, Y)
    if ind % 250 == 0:
        print("ind:",ind)
        testX = np.random.randint(INPUT_SIZE, size=(1, STEP_SIZE))
        test = encode(testX, STEP_SIZE, INPUT_SIZE)
        print("before is")
        print(testX)
        y = model.predict(test, batch_size=1)
        print("actual sorted output is")
        print(np.sort(testX))
        print("sorting done by LSTM is")
        print(np.argmax(y, axis=2))
        print("\n")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
ind: 0
before is
[[ 6  1 11  0 15  6 15  2  9 14  3 14 12 11 18]]
actual sorted output is
[[ 0  1  2  3  6  6  9 11 11 12 14 14 15 15 18]]
sorting done by LSTM is
[[16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]]


ind: 250
before is
[[ 0  2 15  5 14  5 17 13  1 18 14 14  8 12  8]]
actual sorted output is
[[ 0  1  2  5  5  8  8 12 13 14 14 14 15 17 18]]
sorting done by LSTM is
[[ 0  2  4  5  5  8  8  9 13 14 14 15 16 18 19]]


ind: 500
before is
[[13 17 19  2 18  0 16 17 17  2  6 16  3  7 17]]
actual sorted output is
[[ 0  2  2  3  6  7 13 16 16 17 17 17 17 18 19]]
sorting done by LSTM is
[[ 0  1  3  5  7  8 11 13 16 16 17 19 19 19 19]]


ind: 750
before is
[[ 2 18 15 18  1 15 10  5  2 11  2 16 13 16  2]]
actual sorted output is
[[ 1  2  2  2  2  5 10 11 13 15 15 16 16 18 18]]
sorting done by LSTM is
[[ 0  0  3  3  5  6  8 11 13 14 15 15 16 18 18]]


ind: 1000
before is
[[ 7 10 18 14  9 18 19  

KeyboardInterrupt: 

In [16]:
for i in range(10):
    print(i)
print(i)

0
1
2
3
4
5
6
7
8
9
9
